<a href="https://colab.research.google.com/github/Eternal-Spirit/OOPS/blob/main/Object-Oriented%20Inventory%20Management%20System%20for%20%20Tank%20Repairs%20in%20the%20Indian%20Army.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from abc import ABC, abstractmethod

class PartUnavailableError(Exception):
    def __init__(self, part_name):
        self.part_name = part_name
        super().__init__(f"[ERROR] {part_name} not found in any department.")

class InventoryItem:
    def __init__(self, item_id, name, quantity, location):
        self.item_id = item_id
        self.name = name
        self.quantity = quantity
        self.location = location

    def update_quantity(self, change):
        old_quantity = self.quantity
        self.quantity += change
        print(f"[LOG] Item '{self.name}' (ID: {self.item_id}) quantity changed from {old_quantity} to {self.quantity} at location {self.location}.")

class Department:
    def __init__(self, *args, **kwargs):
        self.name = kwargs.pop("name", "Unknown")
        self.items = {}
        super().__init__(*args, **kwargs)

    def add_item(self, item):
        self.items[item.name.lower()] = item

    def order_part(self, part_name, qty):
        key = part_name.lower()
        if key not in self.items:
            return None
        if self.items[key].quantity < qty:
            return None
        self.items[key].update_quantity(-qty)
        return f"[CONFIRM] {qty} {part_name} issued from {self.name}."

    def generate_report(self):
        print(f"\n=== {self.name} Inventory ===")
        for item in self.items.values():
            print(f"{item.name} (ID:{item.item_id}) → Qty: {item.quantity}, Location: {item.location}")

class MechanicalDept(Department):
    def __init__(self, *args, **kwargs):
        kwargs["name"] = "Mechanical"
        super().__init__(*args, **kwargs)
        self.add_item(InventoryItem(201, "Engine", 5, "Warehouse M1"))
        self.add_item(InventoryItem(202, "Tracks", 10, "Warehouse M2"))
        self.add_item(InventoryItem(203, "Suspension", 7, "Warehouse M3"))

class ElectronicsDept(Department):
    def __init__(self, *args, **kwargs):
        kwargs["name"] = "Electronics"
        super().__init__(*args, **kwargs)
        self.add_item(InventoryItem(301, "Radar", 3, "Electronics Bay E1"))
        self.add_item(InventoryItem(302, "Communication System", 8, "Electronics Bay E2"))

class WeaponryDept(Department):
    def __init__(self, *args, **kwargs):
        kwargs["name"] = "Weaponry"
        super().__init__(*args, **kwargs)
        self.add_item(InventoryItem(401, "Cannon", 2, "Armory W1"))
        self.add_item(InventoryItem(402, "Autoloader", 4, "Armory W2"))
        self.add_item(InventoryItem(403, "Targeting System", 6, "Armory W3"))

class TankRepairBase(ABC):
    @abstractmethod
    def allocate_team(self, tank_model, complexity_level):
        pass

    @abstractmethod
    def log_repair(self, tank_model, parts_used):
        pass

class TankRepairUnit(MechanicalDept, ElectronicsDept, WeaponryDept, TankRepairBase):
    repair_history = []

    def __init__(self):
        super().__init__()

    def allocate_team(self, tank_model, complexity_level):
        team_sizes = {"low": 2, "medium": 4, "high": 6}
        team_size = team_sizes.get(complexity_level.lower(), 2)
        print(f"[TEAM] Allocated {team_size} technicians for repairing {tank_model}")
        return team_size

    def log_repair(self, tank_model, parts_used):
        TankRepairUnit.repair_history.append({"tank": tank_model, "parts": parts_used})
        print(f"[LOG] Repair logged for {tank_model}: {parts_used}")

    def order_part(self, part_name, qty):
        key = part_name.lower()
        for cls in TankRepairUnit.__mro__[1:]:
            if hasattr(cls, "items") and key in cls.items:
                return cls.order_part(self, part_name, qty)
        raise PartUnavailableError(part_name)

    def schedule_repair(self, tank_model, issues_list, complexity_level):
        print(f"\n[SCHEDULE] Repair scheduled for {tank_model} with issues: {issues_list}")
        self.allocate_team(tank_model, complexity_level)
        used_parts = []
        for part in issues_list:
            try:
                confirmation = self.order_part(part, 1)
                if confirmation:
                    print(confirmation)
                    used_parts.append(part)
            except PartUnavailableError as e:
                print(e)
        self.log_repair(tank_model, used_parts)

if __name__ == "__main__":
    unit = TankRepairUnit()

    unit.add_item(InventoryItem("M4", "Gearbox", 3, "Mech-Store"))
    unit.add_item(InventoryItem("M5", "Hull", 2, "Mech-Store"))
    unit.add_item(InventoryItem("E3", "Night Vision", 2, "Elec-Store"))
    unit.add_item(InventoryItem("E4", "Sensor", 5, "Elec-Store"))
    unit.add_item(InventoryItem("E5", "Battery", 6, "Elec-Store"))
    unit.add_item(InventoryItem("W4", "Missile Launcher", 1, "Weap-Store"))
    unit.add_item(InventoryItem("W5", "Ammo Box", 5, "Weap-Store"))

    MechanicalDept.generate_report(unit)
    ElectronicsDept.generate_report(unit)
    WeaponryDept.generate_report(unit)

    print("\n--- MRO of TankRepairUnit ---")
    for cls in TankRepairUnit.mro():
        print(cls)

    unit.schedule_repair("Arjun MBT", ["Engine", "Radar"], complexity_level="high")
    unit.schedule_repair("T-90", ["Cannon", "Autoloader"], complexity_level="medium")
    unit.schedule_repair("T-72", ["Tracks", "Communication System", "Targeting System"], complexity_level="high")

    MechanicalDept.generate_report(unit)
    ElectronicsDept.generate_report(unit)
    WeaponryDept.generate_report(unit)

    print("\n--- Repair History ---")
    for record in TankRepairUnit.repair_history:
        print(record)








=== Weaponry Inventory ===
Cannon (ID:401) → Qty: 2, Location: Armory W1
Autoloader (ID:402) → Qty: 4, Location: Armory W2
Targeting System (ID:403) → Qty: 6, Location: Armory W3
Radar (ID:301) → Qty: 3, Location: Electronics Bay E1
Communication System (ID:302) → Qty: 8, Location: Electronics Bay E2
Engine (ID:201) → Qty: 5, Location: Warehouse M1
Tracks (ID:202) → Qty: 10, Location: Warehouse M2
Suspension (ID:203) → Qty: 7, Location: Warehouse M3
Gearbox (ID:M4) → Qty: 3, Location: Mech-Store
Hull (ID:M5) → Qty: 2, Location: Mech-Store
Night Vision (ID:E3) → Qty: 2, Location: Elec-Store
Sensor (ID:E4) → Qty: 5, Location: Elec-Store
Battery (ID:E5) → Qty: 6, Location: Elec-Store
Missile Launcher (ID:W4) → Qty: 1, Location: Weap-Store
Ammo Box (ID:W5) → Qty: 5, Location: Weap-Store

=== Weaponry Inventory ===
Cannon (ID:401) → Qty: 2, Location: Armory W1
Autoloader (ID:402) → Qty: 4, Location: Armory W2
Targeting System (ID:403) → Qty: 6, Location: Armory W3
Radar (ID:301) → Qty: 3, 